<a href="https://colab.research.google.com/github/xarvel/DataScience/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import tensorflow as tf

import numpy as np
import os
import time

In [81]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [139]:
def text_stats(text):
  sample_size = 250
  print(f'Sample {sample_size} characters:')
  print('-' * 80)
  # Take a look at the first 250 characters in text
  print(text[:sample_size])
  print('-' * 80)
  # length of text is the number of characters in it
  print(f'Length of text: {len(text)} characters')
  # The unique characters in the file
  vocab = sorted(set(text))
  print(f'{len(vocab)} unique characters')

  return vocab

In [140]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = text_stats(text)

Sample 250 characters:
--------------------------------------------------------------------------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

--------------------------------------------------------------------------------
Length of text: 1115394 characters
65 unique characters


In [85]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [86]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)

In [87]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [88]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

In [89]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [90]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [91]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [92]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [93]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [94]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [95]:
seq_length = 100

In [96]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [97]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [98]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [99]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [100]:
dataset = sequences.map(split_input_target)

In [101]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [102]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [103]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [104]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [105]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [106]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [107]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  16896     
                                                                 
 gru_2 (GRU)                 multiple                  3938304   
                                                                 
 dense_2 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [108]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [109]:
sampled_indices

array([27,  1, 61, 52, 41, 53, 26, 49,  2, 11, 17, 60,  5, 30, 59, 10, 40,
       10, 19,  8, 65, 52, 27, 33, 52, 41, 29,  7, 55, 54,  4, 19, 62, 33,
       17,  9, 63, 64,  9, 50, 64, 62, 24, 17, 63,  1, 27,  1, 27, 56, 32,
       59, 40, 35, 18, 16, 63, 36, 49, 61, 65, 14, 14, 13,  1, 22, 41, 26,
        9, 58, 30, 35, 52, 24, 59, 62, 42, 38, 62, 23, 21, 62, 41, 65, 59,
        5, 43,  2, 42, 53, 15, 23, 35, 54, 40, 24, 32, 27, 33, 32])

In [110]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'K:\nAbout what?\n\nRICHARD:\nAbout that which concerns your grace and us;\nThe crown of England, father, '

Next Char Predictions:
 b'N\nvmbnMj :Du&Qt3a3F-zmNTmbP,po$FwTD.xy.kywKDx\nN\nNqStaVECxWjvzAA?\nIbM.sQVmKtwcYwJHwbzt&d cnBJVoaKSNTS'


In [111]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [112]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190171, shape=(), dtype=float32)


In [113]:
tf.exp(example_batch_mean_loss).numpy()

66.034065

In [114]:
model.compile(optimizer='adam', loss=loss)

In [115]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [116]:
EPOCHS = 20

In [117]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 16s 61ms/step - loss: 2.7267
Epoch 2/20
172/172 [==============================] - 13s 58ms/step - loss: 1.9970
Epoch 3/20
172/172 [==============================] - 12s 59ms/step - loss: 1.7177
Epoch 4/20
172/172 [==============================] - 12s 61ms/step - loss: 1.5536
Epoch 5/20
172/172 [==============================] - 12s 62ms/step - loss: 1.4543
Epoch 6/20
172/172 [==============================] - 12s 61ms/step - loss: 1.3864
Epoch 7/20
172/172 [==============================] - 11s 58ms/step - loss: 1.3341
Epoch 8/20
172/172 [==============================] - 12s 59ms/step - loss: 1.2900
Epoch 9/20
172/172 [==============================] - 13s 61ms/step - loss: 1.2497
Epoch 10/20
172/172 [==============================] - 12s 62ms/step - loss: 1.2109
Epoch 11/20
172/172 [==============================] - 12s 61ms/step - loss: 1.1734
Epoch 12/20
172/172 [==============================] - 12s 61ms/step - loss: 1.1333
E

In [118]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [119]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [120]:
def predict_text(start_text, size):
  start = time.time()
  states = None
  next_char = tf.constant([start_text])
  result = [next_char]

  for n in range(size):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

  result = tf.strings.join(result)
  end = time.time()
  print(result[0].numpy().decode('utf-8'), '\n\n' + '_' * 80)
  print('\nRun time:', end - start)

In [121]:
predict_text('ROMEO:', 1000)

ROMEO:
Two I chappel that madam, you not: a great man
what I promise you?

FRIAR LAURENCE:
Romeo, there are too sore golden crowns. The
sweeter Shot to do you go,
A save forfeited and sweetly repair,
Do evel to again for right,
Have not done exactive,
So saved the noble duke, unweighing
gods, proud man than dost thou his shrunk.
What you that kill'd her with a toath of tain,
His honour and a closet-fulbert, I might
Thus by false enly's and fortune,
As if they were the life removed out of holy own.
Ah, wouldding the wild and things and honour,
Or your hearing still, and studience against somewhat
to the law; the bridegroom in our intent
Told ourselves that order glory. To him Dorpett
Ir lowly asserve me? if she would prove it other,
And all that love from ungraceful greatest enter
Of tears are in respetite, and make
her passing exercise, I am his,
You must repart forth me to elsche yield
As let it be true.

VOLUMNIA:
I new'd too wife.

LUCENTIO:
All man hear there, even of common grave;

In [122]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThis rusty have I mean on their cheeks,\nDid frame and vow'd down and so outray: may I had a\ndesire to live, in Greece of unstain'd\nsince Duke of York. Foult to himself and king!\nFor Warwick well arrived out of my flesh.\nBe pleased, tremble--as I did lip or\nAngel: for a gloring am at Pomfret.\n\nVIRGILIA:\nO, he is gone.\n\nLEONTES:\nThou art neighbours:\nThe Earl of Warwick will disloy him they house;\nFor, knock joy; then among these woes unto me\nAs is a minute that goveth since\nProve in scorn at once: thrice noble mounh\nTo wedded the air way shame to keep hildly be.\n\nPERDITA:\nNow See where Obbour issue.\n\nBUSHY:\nMy light and the lists of your delights?\nHere is a bod on that advised\nIs as a punking heavy days\nBy admoried their stolen or surlent,--\nMore Pomfret, Catesby, what say it,\nHortensio, that is gone, to prison,\nNot first he crowns in a peinzer.\n\nHORTENSIO:\nSome heavy leave and well.\n\nDUCHESS OF YORK:\nA good morrow's name, I hope.\

In [123]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [124]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The queen with her husband, for prattling friends
Apong a temporaze over'd on my treacherous,
But t


In [125]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [126]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [127]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [128]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 58ms/step - loss: 2.7406


In [129]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.2014
Epoch 1 Batch 50 Loss 2.0813
Epoch 1 Batch 100 Loss 1.9534
Epoch 1 Batch 150 Loss 1.8573

Epoch 1 Loss: 2.0073
Time taken for 1 epoch 13.15 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7937
Epoch 2 Batch 50 Loss 1.7321
Epoch 2 Batch 100 Loss 1.7420
Epoch 2 Batch 150 Loss 1.6268

Epoch 2 Loss: 1.7233
Time taken for 1 epoch 12.02 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6268
Epoch 3 Batch 50 Loss 1.5937
Epoch 3 Batch 100 Loss 1.5560
Epoch 3 Batch 150 Loss 1.5513

Epoch 3 Loss: 1.5566
Time taken for 1 epoch 11.93 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4845
Epoch 4 Batch 50 Loss 1.4787
Epoch 4 Batch 100 Loss 1.4579
Epoch 4 Batch 150 Loss 1.4226

Epoch 4 Loss: 1.4555
Time taken for 1 epoch 11.50 sec
_____________________________________________________________________